<a href="https://colab.research.google.com/github/cipz/AlgoritmiAvanzati/blob/master/Homework2/exactHeldKarp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Held and Karp algorithm

In [125]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.96 s


In [126]:
import math
import numpy as np
# u, v: vertexes/coordinates
# t: WEIGHT_TYPE
def weight (u, v, t):
  if t == "EUC_2D":
    return round(math.sqrt(sum([(a - b) ** 2 for a, b in zip(u, v)])))
  else:
    PI = 3.141592
    deg_xu = int(u[0])
    min_xu = u[0] - deg_xu
    rad_xu = PI * (deg_xu + 5.0 * min_xu/ 3.0) / 180.0

    deg_yu = int(u[1])
    min_yu = u[1] - deg_yu
    rad_yu = PI * (deg_yu + 5.0 * min_yu/ 3.0) / 180.0

    deg_xv = int(v[0])
    min_xv = v[0] - deg_xv
    rad_xv= PI * (deg_xv + 5.0 * min_xv/ 3.0) / 180.0

    deg_yv = int(v[1])
    min_yv = v[1]- deg_yv
    rad_yv = PI * (deg_yv + 5.0 * min_yv/ 3.0) / 180.0

    RRR = 6378.388
    q1 = math.cos(rad_yu - rad_yv)
    q2 = math.cos(rad_xu - rad_xv)
    q3 = math.cos(rad_xu + rad_xv)
    return (int) (RRR * math.acos(0.5 * ((1.0 + q1) * q2 - (1.0 - q1) * q3)) + 1.0)

time: 14.3 ms


In [127]:
lines = open("test.tsp", "r").readlines()
index_start_coordinates = 0
cont = 0
V = []

for line in lines:
  cont += 1
  if line.startswith("EOF"):
    break
  elif line.startswith("DIMENSION"):
    n = int(line.split(":")[1][1:])
  elif line.startswith("EDGE_WEIGHT_TYPE"):
    t = line.split(":")[1][1:]
  elif line.startswith("NODE_COORD_SECTION"):
    index_start_coordinates = cont
  elif (index_start_coordinates > 0):
    V.append((int(line.split()[0]) - 1, [float(line.split()[1]), float(line.split()[2])])) #(i, [x_value, y_value])
#n = int(lines[3].split()[1]) #.split()[0] # extract number of vertexes
#t = lines[4].split()[1]
print(n,t)
'''
for line in lines:
  if line.split()[0] == 'DIMENSION:':
    n = line.split()[0]
  else if line.split()[0] == 'EDGE_WEIGHT_TYPE:'
    t = line.split()[0]
  print()'''

'''V = [] # array of vertexes/coordinates
for i in range(8, 8 + n):
  V.append((int(lines[i].split()[0]) - 1, [float(lines[i].split()[1]), float(lines[i].split()[2])])) #(i, [x_value, y_value])
'''
print(V)

4 GEO

[(0, [16.47, 96.1]), (1, [16.47, 94.44]), (2, [20.09, 92.54]), (3, [22.39, 93.37])]
time: 15.4 ms


In [128]:
# global values

subsets = []
subsets.append(V)
#phi = [[None] * 10] * 10 #[[]]
#d = [[None] * 10] * 10
d = np.zeros(shape = (n, 2 ** (n - 1))) # 2^(n-1)
phi = np.zeros(shape = (n, 2 ** (n - 1)))

def get_subset_index (S):
  cont = 0
  for s in subsets:
    if s == S:
      return cont
    else:
      cont += 1
  return False


time: 5.98 ms


In [129]:
import time
start = time.time()
def HK_VISIT (v, S): # v: arrival vertex of S, S: subset of vertexes 
  S_index = get_subset_index(S)
  if (time.time() - start) > 0.16:
    return 100
  #print(S_index)
  elif (S_index is not False) & (len(S) == 1) & (S[0][0] == v): # Caso base: la soluzione è il peso dell’arco {v, 0}
    #print("1 case")
    return weight(V[v][1], V[0][1], t)
  elif d[v,S_index] != 0: # Distanza già calcolata, ritorna il valore memorizzato
    #print("2 case")
    return d[v,S_index] 
  else:  # Caso ricorsivo: trova il minimo tra tutti i sottocammini
    #print("3 case")
    mindist = math.inf
    minprec = None
    temp = [i for i in S if i[0] != v] # S \ {v} 
    subsets.append(temp) if temp not in subsets else subsets
    for u in temp:
      dist = HK_VISIT(u[0], temp)
      if (dist + weight(u[1], V[v][1], t)) < mindist:
        mindist = dist + weight(u[1], V[v][1], t)
        minprec = u[0]
    #print(v, S_index)
    d[v,S_index] = mindist
    phi[v,S_index] = minprec
    return mindist

time: 20.7 ms


In [130]:
print(HK_VISIT(0, V))
print(time.time() - start , "s")
print(len(subsets))
print(np.matrix(subsets))
#print(subsets)
print(np.matrix(d))

1570
0.011000871658325195 s
8
[[list([(0, [16.47, 96.1]), (1, [16.47, 94.44]), (2, [20.09, 92.54]), (3, [22.39, 93.37])])
  list([(1, [16.47, 94.44]), (2, [20.09, 92.54]), (3, [22.39, 93.37])])
  list([(2, [20.09, 92.54]), (3, [22.39, 93.37])])
  list([(3, [22.39, 93.37])]) list([(2, [20.09, 92.54])])
  list([(1, [16.47, 94.44]), (3, [22.39, 93.37])])
  list([(1, [16.47, 94.44])])
  list([(1, [16.47, 94.44]), (2, [20.09, 92.54])])]]
[[1570.    0.    0.    0.    0.    0.    0.    0.]
 [   0. 1417.    0.    0.    0. 1370.    0.  932.]
 [   0. 1106.  995.    0.    0.    0.    0.  575.]
 [   0.  864.  799.    0.    0.  817.    0.    0.]]
time: 7.95 ms
